In [3]:
#If you need to install in modules in jupyter notebook 
%pip install scikit-learn
%pip install pandas
%pip install matplotlib 
%pip install seaborn 
%pip install graphviz
%pip install scikit-plot   
%pip install statsmodels   
%pip install keras
%pip install tensorflow

#Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)
import datetime # Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"


# Import required libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical 
from tensorflow.keras import layers
from tensorflow.keras import activations
import keras.backend as K
from keras.utils.vis_utils import plot_model
import graphviz


# to make this notebook's output stable across runs
np.random.seed(42)

# Where to save the figures
# PROJECT_ROOT_DIR = "."
# CHAPTER_ID = "decision_trees"
# IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)

# os.makedirs(IMAGES_PATH, exist_ok=True)
# def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):

#     path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
#     print("Saving figure", fig_id)
#     if tight_layout:
#         plt.tight_layout()
#     plt.savefig(path, format=fig_extension, dpi=resolution)

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\shaun\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\shaun\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.



Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\shaun\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


...

You should consider upgrading via the 'c:\users\shaun\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\shaun\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\shaun\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\shaun\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [4]:
#changes the output for the print statements
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth',0)

In [5]:
from functools import reduce
from itertools import chain
import operator 
#function for counting how many units within each layer and how many layers
def count_units(model):
  tot_out = 0
  out_list = []
  for lyr in model.layers:
    if lyr.trainable:
      # This is to tackle any layers that have the output shape as a list of tuples (e.g Input layer)
      if isinstance(lyr.output_shape, list):
        curr_out = reduce(operator.mul, chain(*[s[1:] for s in lyr.output_shape]), 1)
      # This is to tackle other layers like Dense and Conv2D
      elif isinstance(lyr.output_shape, tuple):
        curr_out = reduce(operator.mul, lyr.output_shape[1:], 1)
      else:
        raise TypeError
      tot_out += curr_out
      out_list.append(curr_out)
  print("Total number of output units:",tot_out)
  print("Output units in each layer as a list:",out_list)

In [62]:
#Pull 
df = pd.read_csv('https://raw.githubusercontent.com/Rodrig79/Machine-Learning-Data-Analysis-Project/master/rawData/pokemon.csv') 

In [63]:
from sklearn import preprocessing

#Data Cleaning
df = df[~df.Name.str.contains('Mega')] #removed pokemon with "Mega" in it
df = df.drop(columns = ["Name","Type 1","Type 2","#","Generation"]) #Removed columns with names

In [64]:
target_column = ['Legendary']
predictors = list(set(list(df.columns))-set(target_column))
df[predictors] = df[predictors]/df[predictors].max()
df.describe()

,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
count,751.000000,751.000000,751.000000,751.000000,751.000000,751.000000,751.000000
mean,0.549769,0.269293,0.422104,0.311932,0.390072,0.304823,0.370040
std,0.146666,0.101750,0.166225,0.128826,0.168133,0.119135,0.154698
min,0.233766,0.003922,0.027778,0.021739,0.055556,0.086957,0.027778
25%,0.422078,0.196078,0.305556,0.217391,0.250000,0.217391,0.250000
50%,0.558442,0.254902,0.416667,0.295652,0.361111,0.282609,0.361111
75%,0.649351,0.313725,0.527778,0.391304,0.500000,0.369565,0.491667
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [65]:
X = df[predictors].values
y = df[target_column].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print(X_train.shape); print(X_test.shape)

(525, 7)
(226, 7)


In [66]:
# one hot encode outputs
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

count_classes = y_test.shape[1]
print(count_classes)

2


In [67]:
def stepsPerEpoch(x_train,batch_size):
    steps_per_epoch = int( np.ceil(x_train.shape[0] / batch_size) )
    print("steps per epoch:",steps_per_epoch)

In [68]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

def c_model():
    model = Sequential()
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
batch_size = 32
model = KerasClassifier(build_fn=c_model, epochs=50, batch_size=batch_size)

In [69]:
model.fit(X_train, y_train)


Epoch 1/50
525/525 [==============================] - 1s 981us/step - loss: 0.5314 - accuracy: 0.9124
Epoch 2/50
525/525 [==============================] - 0s 71us/step - loss: 0.4477 - accuracy: 0.9124
Epoch 3/50
525/525 [==============================] - 0s 71us/step - loss: 0.3927 - accuracy: 0.9124
Epoch 4/50
525/525 [==============================] - 0s 63us/step - loss: 0.3759 - accuracy: 0.9124
Epoch 5/50
525/525 [==============================] - 0s 69us/step - loss: 0.3714 - accuracy: 0.9124
Epoch 6/50
525/525 [==============================] - 0s 67us/step - loss: 0.3650 - accuracy: 0.9124
Epoch 7/50
525/525 [==============================] - 0s 69us/step - loss: 0.3568 - accuracy: 0.9124
Epoch 8/50
525/525 [==============================] - 0s 63us/step - loss: 0.3495 - accuracy: 0.9124
Epoch 9/50
525/525 [==============================] - 0s 99us/step - loss: 0.3420 - accuracy: 0.9124
Epoch 10/50
525/525 [==============================] - 0s 80us/step - loss: 0.3350 - accur

In [120]:
stepsPerEpoch(X_train,batch_size) 
print(model.model.summary())
print(count_units(model.model))
model.model.optimizer.get_config()

steps per epoch: 17
Model: "sequential_170"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_508 (Dense)            (None, 32)                256       
_________________________________________________________________
dense_509 (Dense)            (None, 16)                528       
_________________________________________________________________
dense_510 (Dense)            (None, 2)                 34        
Total params: 818
Trainable params: 818
Non-trainable params: 0
_________________________________________________________________
None
Total number of output units: 50
Output units in each layer as a list: [32, 16, 2]
None


{'learning_rate': 0.0010000000474974513,
 'beta_1': 0.8999999761581421,
 'beta_2': 0.9990000128746033,
 'decay': 0.0,
 'epsilon': 1e-07,
 'amsgrad': False}

In [71]:
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=c_model)

batch_sizes = [10, 20, 50, 100]
epochs = [5, 10, 50]
parameters = {'batch_size': batch_sizes, 'epochs': epochs}
clf = GridSearchCV(model, parameters)
clf.fit(X_train, y_train)

Epoch 1/5
420/420 [==============================] - 1s 1ms/step - loss: 0.5531 - accuracy: 0.9071
Epoch 2/5
420/420 [==============================] - 0s 200us/step - loss: 0.3800 - accuracy: 0.9143
Epoch 3/5
420/420 [==============================] - 0s 179us/step - loss: 0.3550 - accuracy: 0.9143
Epoch 4/5
420/420 [==============================] - 0s 183us/step - loss: 0.3445 - accuracy: 0.9143
Epoch 5/5
105/105 [==============================] - 1s 5ms/step
Epoch 1/5
420/420 [==============================] - 1s 1ms/step - loss: 0.6662 - accuracy: 0.6643
Epoch 2/5
420/420 [==============================] - 0s 204us/step - loss: 0.4664 - accuracy: 0.9167
Epoch 3/5
420/420 [==============================] - 0s 226us/step - loss: 0.3546 - accuracy: 0.9167
Epoch 4/5
420/420 [==============================] - 0s 183us/step - loss: 0.3430 - accuracy: 0.9167
Epoch 5/5
105/105 [==============================] - 0s 4ms/step
Epoch 1/5
420/420 [==============================] - 1s 1ms/step -

420/420 [==============================] - 1s 2ms/step - loss: 0.5603 - accuracy: 0.9048
Epoch 2/50
420/420 [==============================] - 0s 202us/step - loss: 0.3968 - accuracy: 0.9143
Epoch 3/50
420/420 [==============================] - 0s 221us/step - loss: 0.3637 - accuracy: 0.9143
Epoch 4/50
420/420 [==============================] - 0s 281us/step - loss: 0.3546 - accuracy: 0.9143
Epoch 5/50
420/420 [==============================] - 0s 201us/step - loss: 0.3467 - accuracy: 0.9143
Epoch 6/50
420/420 [==============================] - 0s 200us/step - loss: 0.3393 - accuracy: 0.9143
Epoch 7/50
420/420 [==============================] - 0s 206us/step - loss: 0.3274 - accuracy: 0.9143
Epoch 8/50
420/420 [==============================] - 0s 197us/step - loss: 0.3127 - accuracy: 0.9143
Epoch 9/50
420/420 [==============================] - 0s 223us/step - loss: 0.2964 - accuracy: 0.9143
Epoch 10/50
420/420 [==============================] - 0s 199us/step - loss: 0.2819 - accuracy:

420/420 [==============================] - 0s 209us/step - loss: 0.1009 - accuracy: 0.9619
Epoch 31/50
420/420 [==============================] - 0s 190us/step - loss: 0.0892 - accuracy: 0.9690
Epoch 32/50
420/420 [==============================] - 0s 193us/step - loss: 0.0886 - accuracy: 0.9762
Epoch 33/50
420/420 [==============================] - 0s 188us/step - loss: 0.0907 - accuracy: 0.9571
Epoch 34/50
420/420 [==============================] - 0s 185us/step - loss: 0.0847 - accuracy: 0.9619
Epoch 35/50
420/420 [==============================] - 0s 202us/step - loss: 0.0834 - accuracy: 0.9738
Epoch 36/50
420/420 [==============================] - 0s 303us/step - loss: 0.0837 - accuracy: 0.9690
Epoch 37/50
420/420 [==============================] - 0s 197us/step - loss: 0.0771 - accuracy: 0.9643
Epoch 38/50
420/420 [==============================] - 0s 195us/step - loss: 0.0723 - accuracy: 0.9762
Epoch 39/50
420/420 [==============================] - 0s 197us/step - loss: 0.0755 -

420/420 [==============================] - 0s 260us/step - loss: 0.2927 - accuracy: 0.9262
Epoch 9/50
420/420 [==============================] - 0s 282us/step - loss: 0.2822 - accuracy: 0.9262
Epoch 10/50
420/420 [==============================] - 0s 271us/step - loss: 0.2704 - accuracy: 0.9262
Epoch 11/50
420/420 [==============================] - 0s 246us/step - loss: 0.2614 - accuracy: 0.9262
Epoch 12/50
420/420 [==============================] - 0s 277us/step - loss: 0.2404 - accuracy: 0.9262
Epoch 13/50
420/420 [==============================] - 0s 307us/step - loss: 0.2220 - accuracy: 0.9262
Epoch 14/50
420/420 [==============================] - 0s 240us/step - loss: 0.2061 - accuracy: 0.9262
Epoch 15/50
420/420 [==============================] - 0s 349us/step - loss: 0.1877 - accuracy: 0.9262
Epoch 16/50
420/420 [==============================] - 0s 298us/step - loss: 0.1676 - accuracy: 0.9262
Epoch 17/50
420/420 [==============================] - 0s 217us/step - loss: 0.1632 - 

420/420 [==============================] - 0s 214us/step - loss: 0.0921 - accuracy: 0.9571
Epoch 37/50
420/420 [==============================] - 0s 198us/step - loss: 0.0868 - accuracy: 0.9667
Epoch 38/50
420/420 [==============================] - 0s 188us/step - loss: 0.0913 - accuracy: 0.9595
Epoch 39/50
420/420 [==============================] - 0s 345us/step - loss: 0.0815 - accuracy: 0.9667
Epoch 40/50
420/420 [==============================] - 0s 190us/step - loss: 0.0791 - accuracy: 0.9667
Epoch 41/50
420/420 [==============================] - 0s 212us/step - loss: 0.0792 - accuracy: 0.9643
Epoch 42/50
420/420 [==============================] - 0s 198us/step - loss: 0.0761 - accuracy: 0.9643
Epoch 43/50
420/420 [==============================] - 0s 193us/step - loss: 0.0763 - accuracy: 0.9738
Epoch 44/50
420/420 [==============================] - 0s 205us/step - loss: 0.0761 - accuracy: 0.9619
Epoch 45/50
420/420 [==============================] - 0s 205us/step - loss: 0.0862 -

...

420/420 [==============================] - 0s 65us/step - loss: 0.3330 - accuracy: 0.9143
Epoch 40/50
420/420 [==============================] - 0s 65us/step - loss: 0.3303 - accuracy: 0.9143
Epoch 41/50
420/420 [==============================] - 0s 57us/step - loss: 0.3275 - accuracy: 0.9143
Epoch 42/50
420/420 [==============================] - 0s 48us/step - loss: 0.3256 - accuracy: 0.9143
Epoch 43/50
420/420 [==============================] - 0s 50us/step - loss: 0.3229 - accuracy: 0.9143
Epoch 44/50
420/420 [==============================] - 0s 57us/step - loss: 0.3208 - accuracy: 0.9143
Epoch 45/50
420/420 [==============================] - 0s 49us/step - loss: 0.3182 - accuracy: 0.9143
Epoch 46/50
420/420 [==============================] - 0s 66us/step - loss: 0.3157 - accuracy: 0.9143
Epoch 47/50
420/420 [==============================] - 0s 58us/step - loss: 0.3134 - accuracy: 0.9143
Epoch 48/50
420/420 [==============================] - 0s 75us/step - loss: 0.3106 - accuracy:

420/420 [==============================] - 0s 56us/step - loss: 0.3828 - accuracy: 0.8976
Epoch 19/50
420/420 [==============================] - 0s 58us/step - loss: 0.3792 - accuracy: 0.8976
Epoch 20/50
420/420 [==============================] - 0s 56us/step - loss: 0.3754 - accuracy: 0.8976
Epoch 21/50
420/420 [==============================] - 0s 57us/step - loss: 0.3706 - accuracy: 0.8976
Epoch 22/50
420/420 [==============================] - 0s 59us/step - loss: 0.3668 - accuracy: 0.8976
Epoch 23/50
420/420 [==============================] - 0s 54us/step - loss: 0.3615 - accuracy: 0.8976
Epoch 24/50
420/420 [==============================] - 0s 54us/step - loss: 0.3574 - accuracy: 0.8976
Epoch 25/50
420/420 [==============================] - 0s 56us/step - loss: 0.3532 - accuracy: 0.8976
Epoch 26/50
420/420 [==============================] - 0s 60us/step - loss: 0.3489 - accuracy: 0.8976
Epoch 27/50
420/420 [==============================] - 0s 57us/step - loss: 0.3432 - accuracy:

Epoch 48/50
420/420 [==============================] - 0s 49us/step - loss: 0.2278 - accuracy: 0.9262
Epoch 49/50
420/420 [==============================] - 0s 68us/step - loss: 0.2232 - accuracy: 0.9262
Epoch 50/50
105/105 [==============================] - 1s 9ms/step
Epoch 1/50
420/420 [==============================] - 1s 3ms/step - loss: 0.8467 - accuracy: 0.0929
Epoch 2/50
420/420 [==============================] - 0s 56us/step - loss: 0.7755 - accuracy: 0.0929
Epoch 3/50
420/420 [==============================] - 0s 58us/step - loss: 0.7196 - accuracy: 0.2024
Epoch 4/50
420/420 [==============================] - 0s 57us/step - loss: 0.6840 - accuracy: 0.7071
Epoch 5/50
420/420 [==============================] - 0s 58us/step - loss: 0.6568 - accuracy: 0.9000
Epoch 6/50
420/420 [==============================] - 0s 58us/step - loss: 0.6314 - accuracy: 0.9048
Epoch 7/50
420/420 [==============================] - 0s 57us/step - loss: 0.6067 - accuracy: 0.9048
Epoch 8/50
420/420 [===

525/525 [==============================] - 0s 188us/step - loss: 0.1155 - accuracy: 0.9562
Epoch 28/50
525/525 [==============================] - 0s 209us/step - loss: 0.1117 - accuracy: 0.9600
Epoch 29/50
525/525 [==============================] - 0s 166us/step - loss: 0.1067 - accuracy: 0.9581
Epoch 30/50
525/525 [==============================] - 0s 181us/step - loss: 0.1081 - accuracy: 0.9619
Epoch 31/50
525/525 [==============================] - 0s 177us/step - loss: 0.1051 - accuracy: 0.9524
Epoch 32/50
525/525 [==============================] - 0s 178us/step - loss: 0.1052 - accuracy: 0.9524
Epoch 33/50
525/525 [==============================] - 0s 185us/step - loss: 0.0957 - accuracy: 0.9619
Epoch 34/50
525/525 [==============================] - 0s 185us/step - loss: 0.0938 - accuracy: 0.9600
Epoch 35/50
525/525 [==============================] - 0s 216us/step - loss: 0.0887 - accuracy: 0.9638
Epoch 36/50
525/525 [==============================] - 0s 229us/step - loss: 0.0913 -

GridSearchCV(estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x00000161A4FC9898>,
             param_grid={'batch_size': [10, 20, 50, 100],
                         'epochs': [5, 10, 50]})

In [72]:
print(clf.best_score_, clf.best_params_)
means = clf.cv_results_['mean_test_score']
parameters = clf.cv_results_['params']
for mean, parammeter in zip(means, parameters):
    print(mean, parammeter)

0.9657142877578735 {'batch_size': 20, 'epochs': 50}
0.9123809576034546 {'batch_size': 10, 'epochs': 5}
0.9123809576034546 {'batch_size': 10, 'epochs': 10}
0.9561904788017273 {'batch_size': 10, 'epochs': 50}
0.9123809576034546 {'batch_size': 20, 'epochs': 5}
0.9123809576034546 {'batch_size': 20, 'epochs': 10}
0.9657142877578735 {'batch_size': 20, 'epochs': 50}
0.9123809576034546 {'batch_size': 50, 'epochs': 5}
0.9123809576034546 {'batch_size': 50, 'epochs': 10}
0.9561904788017273 {'batch_size': 50, 'epochs': 50}
0.9123809576034546 {'batch_size': 100, 'epochs': 5}
0.9123809576034546 {'batch_size': 100, 'epochs': 10}
0.9123809576034546 {'batch_size': 100, 'epochs': 50}


In [75]:
def c_model(activation):
    model = Sequential()
    model.add(Dense(32, activation=activation))
    model.add(Dense(16, activation=activation))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=c_model, epochs=50, batch_size=32)
parameters = {'activation':['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']}
clf = GridSearchCV(model, parameters)
clf.fit(X_train, y_train)

Epoch 1/50
420/420 [==============================] - 1s 2ms/step - loss: 0.6861 - accuracy: 0.7238 
Epoch 2/50
420/420 [==============================] - 0s 137us/step - loss: 0.6592 - accuracy: 0.9143
Epoch 3/50
420/420 [==============================] - 0s 126us/step - loss: 0.6338 - accuracy: 0.9143
Epoch 4/50
420/420 [==============================] - 0s 152us/step - loss: 0.6097 - accuracy: 0.9143
Epoch 5/50
420/420 [==============================] - 0s 126us/step - loss: 0.5866 - accuracy: 0.9143
Epoch 6/50
420/420 [==============================] - 0s 121us/step - loss: 0.5653 - accuracy: 0.9143
Epoch 7/50
420/420 [==============================] - 0s 158us/step - loss: 0.5455 - accuracy: 0.9143
Epoch 8/50
420/420 [==============================] - 0s 123us/step - loss: 0.5272 - accuracy: 0.9143
Epoch 9/50
420/420 [==============================] - 0s 124us/step - loss: 0.5104 - accuracy: 0.9143
Epoch 10/50
420/420 [==============================] - 0s 127us/step - loss: 0.4939

420/420 [==============================] - 0s 126us/step - loss: 0.3484 - accuracy: 0.9167
Epoch 31/50
420/420 [==============================] - 0s 127us/step - loss: 0.3435 - accuracy: 0.9167
Epoch 32/50
420/420 [==============================] - 0s 135us/step - loss: 0.3390 - accuracy: 0.9167
Epoch 33/50
420/420 [==============================] - 0s 164us/step - loss: 0.3348 - accuracy: 0.9167
Epoch 34/50
420/420 [==============================] - 0s 170us/step - loss: 0.3313 - accuracy: 0.9167
Epoch 35/50
420/420 [==============================] - 0s 113us/step - loss: 0.3279 - accuracy: 0.9167
Epoch 36/50
420/420 [==============================] - 0s 119us/step - loss: 0.3245 - accuracy: 0.9167
Epoch 37/50
420/420 [==============================] - 0s 152us/step - loss: 0.3215 - accuracy: 0.9167
Epoch 38/50
420/420 [==============================] - 0s 132us/step - loss: 0.3185 - accuracy: 0.9167
Epoch 39/50
420/420 [==============================] - 0s 145us/step - loss: 0.3160 -

420/420 [==============================] - 0s 117us/step - loss: 0.5512 - accuracy: 0.9262
Epoch 9/50
420/420 [==============================] - 0s 114us/step - loss: 0.5324 - accuracy: 0.9262
Epoch 10/50
420/420 [==============================] - 0s 116us/step - loss: 0.5153 - accuracy: 0.9262
Epoch 11/50
420/420 [==============================] - 0s 112us/step - loss: 0.4988 - accuracy: 0.9262
Epoch 12/50
420/420 [==============================] - 0s 136us/step - loss: 0.4835 - accuracy: 0.9262
Epoch 13/50
420/420 [==============================] - 0s 117us/step - loss: 0.4694 - accuracy: 0.9262
Epoch 14/50
420/420 [==============================] - 0s 120us/step - loss: 0.4557 - accuracy: 0.9262
Epoch 15/50
420/420 [==============================] - 0s 116us/step - loss: 0.4428 - accuracy: 0.9262
Epoch 16/50
420/420 [==============================] - 0s 119us/step - loss: 0.4312 - accuracy: 0.9262
Epoch 17/50
420/420 [==============================] - 0s 117us/step - loss: 0.4205 - 

420/420 [==============================] - 0s 114us/step - loss: 0.3350 - accuracy: 0.9071
Epoch 38/50
420/420 [==============================] - 0s 119us/step - loss: 0.3327 - accuracy: 0.9071
Epoch 39/50
420/420 [==============================] - 0s 119us/step - loss: 0.3303 - accuracy: 0.9071
Epoch 40/50
420/420 [==============================] - 0s 119us/step - loss: 0.3283 - accuracy: 0.9071
Epoch 41/50
420/420 [==============================] - 0s 112us/step - loss: 0.3265 - accuracy: 0.9071
Epoch 42/50
420/420 [==============================] - 0s 117us/step - loss: 0.3248 - accuracy: 0.9071
Epoch 43/50
420/420 [==============================] - 0s 115us/step - loss: 0.3233 - accuracy: 0.9071
Epoch 44/50
420/420 [==============================] - 0s 119us/step - loss: 0.3218 - accuracy: 0.9071
Epoch 45/50
420/420 [==============================] - 0s 112us/step - loss: 0.3206 - accuracy: 0.9071
Epoch 46/50
420/420 [==============================] - 0s 121us/step - loss: 0.3195 -

420/420 [==============================] - 0s 121us/step - loss: 0.2422 - accuracy: 0.9167
Epoch 16/50
420/420 [==============================] - 0s 133us/step - loss: 0.2376 - accuracy: 0.9167
Epoch 17/50
420/420 [==============================] - 0s 124us/step - loss: 0.2353 - accuracy: 0.9167
Epoch 18/50
420/420 [==============================] - 0s 125us/step - loss: 0.2320 - accuracy: 0.9167
Epoch 19/50
420/420 [==============================] - 0s 136us/step - loss: 0.2293 - accuracy: 0.9167
Epoch 20/50
420/420 [==============================] - 0s 129us/step - loss: 0.2237 - accuracy: 0.9167
Epoch 21/50
420/420 [==============================] - 0s 132us/step - loss: 0.2204 - accuracy: 0.9167
Epoch 22/50
420/420 [==============================] - 0s 139us/step - loss: 0.2156 - accuracy: 0.9167
Epoch 23/50
420/420 [==============================] - 0s 119us/step - loss: 0.2130 - accuracy: 0.9167
Epoch 24/50
420/420 [==============================] - 0s 123us/step - loss: 0.2057 -

...

420/420 [==============================] - 0s 185us/step - loss: 0.0903 - accuracy: 0.9738
Epoch 50/50
105/105 [==============================] - 2s 17ms/step
Epoch 1/50
420/420 [==============================] - 2s 4ms/step - loss: 0.5452 - accuracy: 0.8667
Epoch 2/50
420/420 [==============================] - 0s 193us/step - loss: 0.3824 - accuracy: 0.9167
Epoch 3/50
420/420 [==============================] - 0s 214us/step - loss: 0.3694 - accuracy: 0.9167
Epoch 4/50
420/420 [==============================] - 0s 200us/step - loss: 0.3641 - accuracy: 0.9167
Epoch 5/50
420/420 [==============================] - 0s 174us/step - loss: 0.3565 - accuracy: 0.9167
Epoch 6/50
420/420 [==============================] - 0s 186us/step - loss: 0.3521 - accuracy: 0.9167
Epoch 7/50
420/420 [==============================] - ETA: 0s - loss: 0.3517 - accuracy: 0.91 - 0s 176us/step - loss: 0.3478 - accuracy: 0.9167
Epoch 8/50
420/420 [==============================] - 0s 155us/step - loss: 0.3407 - ac

420/420 [==============================] - 0s 200us/step - loss: 0.1821 - accuracy: 0.9167
Epoch 28/50
420/420 [==============================] - 0s 179us/step - loss: 0.1747 - accuracy: 0.9381
Epoch 29/50
420/420 [==============================] - 0s 174us/step - loss: 0.1696 - accuracy: 0.9262
Epoch 30/50
420/420 [==============================] - 0s 190us/step - loss: 0.1683 - accuracy: 0.9476
Epoch 31/50
420/420 [==============================] - 0s 190us/step - loss: 0.1611 - accuracy: 0.9357
Epoch 32/50
420/420 [==============================] - 0s 202us/step - loss: 0.1563 - accuracy: 0.9429
Epoch 33/50
420/420 [==============================] - 0s 188us/step - loss: 0.1506 - accuracy: 0.9429
Epoch 34/50
420/420 [==============================] - 0s 183us/step - loss: 0.1465 - accuracy: 0.9405
Epoch 35/50
420/420 [==============================] - 0s 188us/step - loss: 0.1429 - accuracy: 0.9381
Epoch 36/50
420/420 [==============================] - 0s 186us/step - loss: 0.1390 -

420/420 [==============================] - 0s 224us/step - loss: 0.3764 - accuracy: 0.9071
Epoch 6/50
420/420 [==============================] - 0s 188us/step - loss: 0.3717 - accuracy: 0.9071
Epoch 7/50
420/420 [==============================] - 0s 186us/step - loss: 0.3661 - accuracy: 0.9071
Epoch 8/50
420/420 [==============================] - 0s 182us/step - loss: 0.3612 - accuracy: 0.9071
Epoch 9/50
420/420 [==============================] - 0s 205us/step - loss: 0.3577 - accuracy: 0.9071
Epoch 10/50
420/420 [==============================] - 0s 205us/step - loss: 0.3508 - accuracy: 0.9071
Epoch 11/50
420/420 [==============================] - 0s 195us/step - loss: 0.3447 - accuracy: 0.9071
Epoch 12/50
420/420 [==============================] - 0s 198us/step - loss: 0.3384 - accuracy: 0.9071
Epoch 13/50
420/420 [==============================] - 0s 192us/step - loss: 0.3319 - accuracy: 0.9071
Epoch 14/50
420/420 [==============================] - 0s 180us/step - loss: 0.3249 - acc

525/525 [==============================] - 0s 173us/step - loss: 0.1199 - accuracy: 0.9543
Epoch 35/50
525/525 [==============================] - 0s 171us/step - loss: 0.1152 - accuracy: 0.9543
Epoch 36/50
525/525 [==============================] - 0s 183us/step - loss: 0.1107 - accuracy: 0.9562
Epoch 37/50
525/525 [==============================] - 0s 216us/step - loss: 0.1071 - accuracy: 0.9619
Epoch 38/50
525/525 [==============================] - 0s 166us/step - loss: 0.1027 - accuracy: 0.9600
Epoch 39/50
525/525 [==============================] - 0s 170us/step - loss: 0.0997 - accuracy: 0.9657
Epoch 40/50
525/525 [==============================] - 0s 175us/step - loss: 0.0986 - accuracy: 0.9562
Epoch 41/50
525/525 [==============================] - 0s 166us/step - loss: 0.0931 - accuracy: 0.9600
Epoch 42/50
525/525 [==============================] - 0s 170us/step - loss: 0.0901 - accuracy: 0.9676
Epoch 43/50
525/525 [==============================] - 0s 168us/step - loss: 0.0895 -

GridSearchCV(estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x0000016226E92BA8>,
             param_grid={'activation': ['softmax', 'softplus', 'softsign',
                                        'relu', 'tanh', 'sigmoid',
                                        'hard_sigmoid', 'linear']})

In [77]:
print(clf.best_score_, clf.best_params_)
means = clf.cv_results_['mean_test_score']
parameters = clf.cv_results_['params']
for mean, parammeter in zip(means, parameters):
    print(mean, parammeter)

0.969523811340332 {'activation': 'linear'}
0.9123809576034546 {'activation': 'softmax'}
0.9447619080543518 {'activation': 'softplus'}
0.9485714316368103 {'activation': 'softsign'}
0.9600000023841858 {'activation': 'relu'}
0.9504761934280396 {'activation': 'tanh'}
0.9123809576034546 {'activation': 'sigmoid'}
0.9123809576034546 {'activation': 'hard_sigmoid'}
0.969523811340332 {'activation': 'linear'}
